In [3]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [6]:
lines1=pd.read_table('/content/Quran-EN.txt', names=['eng'])
lines2=pd.read_table('/content/Quran-UR.txt', names=['urdu'])

In [7]:
lines1['c'] = np.arange(len(lines1))
lines2['c'] = np.arange(len(lines2))

In [8]:
lines = pd.merge(lines1, lines2, on='c', how='outer', suffixes=('en', 'urd'))

In [9]:
del lines['c']

In [10]:
lines.head()

,eng,urdu
0,"All praise be to Allah alone , the Sustainer o...",سب تعریفیں اللہ ہی کے لئے ہیں جو تمام جہانوں ک...
1,"Most Compassionate , Ever - Merciful .",نہایت مہربان بہت رحم فرمانے والا ہے ۔
2,Master of the Day of Judgment .,روزِ جزا کا مالک ہے ۔
3,( O Allah ! ) You alone do we worship and to Y...,اے اللہ ! ہم تیری ہی عبادت کرتے ہیں اور ہم تجھ...
4,Show us the straight path .,ہمیں سیدھا راستہ دکھا ۔


In [11]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.urdu=lines.urdu.apply(lambda x: x.lower())

# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.urdu=lines.urdu.apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation) # Set of all special characters

# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.urdu=lines.urdu.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))

# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.urdu=lines.urdu.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.urdu=lines.urdu.apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines.urdu = lines.urdu.apply(lambda x : 'START_ '+ x + ' _END')

lines.sample(10)

,eng,urdu
536,these are the revelations of allah which we re...,START_ یہ اللہ کی آیتیں ہیں جنہیں ہم آپ پر حق ...
3162,and he is the one who provides me with food an...,START_ اور وہی ہے جو مجھے کھلاتا اور پلاتا ہے ...
4075,that is how we pay back those who are committe...,START_ بے شک ہم نیکو کاروں کو اسی طرح صِلہ دیا...
5983,and when souls will be united with their bodies,START_ اور جب روحیں بدنوں سے ملا دی جائیں گی ۔...
2053,and he is the one who has created firm mountai...,START_ اور اسی نے زمین میں مختلف مادوں کو باہم...
3620,so be steadfast verily the promise of allah is...,START_ پس آپ صبر کیجئے ، بیشک اﷲ کا وعدہ سچا ہ...
6375,have you seen him who denies din religion,START_ کیا آپ نے اس شخص کو دیکھا جو دین کو جھٹ...
3423,and a man came running from the far end of the...,START_ اور شہر کے آخری کنارے سے ایک شخص دوڑتا ...
4855,receiving those blessings in ecstatic delight ...,START_ اُن نعمتوں کو کیف و سرور سے لیتے ہوں گے...
3211,do you build a monument on every high point me...,START_ کیا تم ہر اونچی جگہ پر ایک یادگار تعمیر...


In [12]:

# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of Urdu 
all_urdu_words=set()
for urdu in lines.urdu:
    for word in urdu.split():
        if word not in all_urdu_words:
            all_urdu_words.add(word)



In [13]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print('Max Source Length:',max_length_src)

# Max Length of target sequence
lenght_list=[]
for l in lines.urdu:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
print('Max Target Lenght:',max_length_tar)


Max Source Length: 200
Max Target Lenght: 234


In [14]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_urdu_words))
num_encoder_tokens = len(all_eng_words)+1
num_decoder_tokens = len(all_urdu_words)+1
num_encoder_tokens, num_decoder_tokens

(8117, 8136)

In [15]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

8137

In [16]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

lines = shuffle(lines)
lines.head(10)

,eng,urdu
4233,well can you save him for whom the command of ...,START_ بھلا جس شخص پر عذاب کا حکم ثابت ہو چکا ...
3474,say just answer if allah prolongs the night ov...,START_ فرما دیجئے ذرا اتنا بتاؤ کہ اگر اللہ تم...
3012,say he allah has sent down this quran who know...,START_ فرما دیجئے اِس قرآن کو اُس اللہ نے نازل...
3965,it will be said yes this is the same day of ju...,START_ کہا جائے گا ہاں یہ وہی فیصلہ کا دن ہے ج...
2482,and each one of them has to appear before his ...,START_ اور ان میں سے ہر ایک قیامت کے دن اس کے ...
4193,say that day of rising is the tremendous piece...,START_ فرما دیجئے وہ قیامت بہت بڑی خبر ہے ۔ _END
2718,surely you and the idols you worship besides a...,START_ بیشک تم اور وہ بت جن کی تم اﷲ کے سوا پر...
3028,the day when they will see the angels there wi...,START_ جس دن وہ فرشتوں کو دیکھیں گے تو اس دن م...
4740,and we shall certainly try you till we bring t...,START_ اور ہم ضرور تمہاری آزمائش کریں گے یہاں ...
1179,the rebellious and arrogant chiefs and the cap...,START_ ان کی قوم کے سرداروں اور رئیسوں نے جو س...


**Creating training and test dataset**


In [17]:
# Train - Test Split
X, y = lines.eng, lines.urdu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape


((5772,), (642,))

In [18]:
#Save the train and test dataframes for reproducing the results later, as they are shuffled.

X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [19]:
def generate_batch(X = X_train, y = y_train, batch_size = 64):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [20]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 100
latent_dim = 256
print(train_samples//batch_size)
print(val_samples//batch_size)

90
10


In [21]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [22]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [23]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [24]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [25]:
model.fit(generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/100
90/90 [==============================] - 53s 404ms/step - loss: 1.0061 - acc: 0.0639 - val_loss: 0.9559 - val_acc: 0.0858
Epoch 2/100
90/90 [==============================] - 34s 372ms/step - loss: 0.8858 - acc: 0.1187 - val_loss: 0.8800 - val_acc: 0.1431
Epoch 3/100
90/90 [==============================] - 37s 413ms/step - loss: 0.8215 - acc: 0.1560 - val_loss: 0.8322 - val_acc: 0.1723
Epoch 4/100
90/90 [==============================] - 36s 403ms/step - loss: 0.7771 - acc: 0.1891 - val_loss: 0.7997 - val_acc: 0.1978
Epoch 5/100
90/90 [==============================] - 37s 407ms/step - loss: 0.7424 - acc: 0.2116 - val_loss: 0.7747 - val_acc: 0.2120
Epoch 6/100
90/90 [==============================] - 37s 408ms/step - loss: 0.7141 - acc: 0.2280 - val_loss: 0.7543 - val_acc: 0.2255
Epoch 7/100
90/90 [==============================] - 36s 406ms/step - loss: 0.6903 - acc: 0.2427 - val_loss: 0.7375 - val_acc: 0.2396
Epoch 8/100
90/90 [==============================] - 37s 412ms

In [26]:
#Always remember to save the weights

model.save_weights('etu_weights.h5')

In [27]:
#Load the weights, if you close the application

model.load_weights('etu_weights.h5')

In [28]:
#Inference Setup

# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [29]:
#Decode sample sequeces

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence



**Evaluation on Train Dataset**

In [30]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [34]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Urdu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:])

1/1 [==============================] - 0s 19ms/step
Input English sentence: he gets upset and worried when some hardship or financial loss afflicts him
Actual Urdu Translation:  جب اسے مصیبت یا مالی نقصان پہنچے تو گھبرا جاتا ہے ۔ 
Predicted Urdu Translation:  جب اسے مصیبت یا مالی نقصان پہنچے تو گھبرا جاتا ہے ۔


In [35]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Urdu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:])

1/1 [==============================] - 0s 16ms/step
Input English sentence: and all their evil works will be exposed which they used to do and that torment will beset them which they used to make fun of
Actual Urdu Translation:  اور اُن کے لئے وہ سب برائیاں ظاہر ہو جائیں گی جو وہ انجام دیتے تھے اور وہ عذاب انہیں گھیر لے گا جِس کا وہ مذاق اڑایا کرتے تھے ۔ 
Predicted Urdu Translation:  اور اُن کے لئے وہ سب برائیاں ظاہر ہو وہ جو ہمیشہ رہے


In [33]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Urdu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 21ms/step
Input English sentence: o believers when you whisper among yourselves whisper not sin injustice revolt and disobedience to the holy prophet blessings and peace be upon him but whisper piety and godwariness and keep fearing allah towards whom you all will be gathered
Actual Urdu Translation:  اے ایمان والو جب تم آپس میں سرگوشی کرو تو گناہ اور ظلم و سرکشی اور نافرمانئ رسالت مآب صلی اللہ علیہ وآلہ وسلم کی سرگوشی نہ کیا کرو اور نیکی اور پرہیزگاری کی بات ایک دوسرے کے کان میں کہہ لیا کرو ، اور اللہ سے ڈرتے رہو جس کی طرف تم سب جمع کئے جاؤ گے ۔ 
Predicted Urdu Translation:  اے ایمان والو جب تم آپس میں سرگوشی کرو تو گناہ


**Evaluation on Validation Dataset**

In [36]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [37]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Urdu Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 19ms/step
Input English sentence: and that he alone is the lord of sirius a star that was worshipped in the days of ignorance
Actual Urdu Translation:  اور یہ کہ وہی شِعرٰی ستارے کا رب ہے جس کی دورِ جاہلیت میں پوجا کی جاتی تھی ۔ 
Predicted Urdu Translation:  اور یہ کہ اسی نے ہر نفس کے ساتھ آرام کر لیا او


In [38]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Urdu Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 20ms/step
Input English sentence: indeed those who believe and keep doing pious actions it is they who are the best of the creation
Actual Urdu Translation:  بیشک جو لوگ ایمان لائے اور نیک عمل کرتے رہے وہی لوگ ساری مخلوق سے بہتر ہیں ۔ 
Predicted Urdu Translation:  بیشک جو لوگ ایمان لائے اور نیک عمل کرتے رہے ان ل


In [39]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Urdu Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 17ms/step
Input English sentence: so which of your lords blessings will you both deny
Actual Urdu Translation:  پس تم دونوں اپنے رب کی کن کن نعمتوں کو جھٹلاؤ گے ۔ 
Predicted Urdu Translation:  پس تم دونوں اپنے رب کی کن کن نعمتوں کو جھٹلاؤ 


In [40]:
from nltk.translate.bleu_score import sentence_bleu

In [44]:
reference = [
    y_test[k:k+1].values[0][6:-4].split()
]
print(reference)

[['پس', 'تم', 'دونوں', 'اپنے', 'رب', 'کی', 'کن', 'کن', 'نعمتوں', 'کو', 'جھٹلاؤ', 'گے', '۔']]


In [45]:
candidate = decoded_sentence[:-4].split()
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate)))

BLEU score -> 0.8337529180751805
